In [10]:
import sys
sys.path.append("../")
from os.path import join
from IPython.display import display
import gc
import json

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rc('text', usetex=False)

import matplotlib.pyplot as plt
import joblib
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm
import faiss

from lolip.models.torch_utils.archs import *
from lolip.variables import auto_var
from params import *
from utils import params_to_dataframe
from notebook_utils import get_model_name, get_arch, get_ds_name

fontsize=15
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
def get_file_paths(ds_name, arch_name="WRN_40_10"):
    if "pathmnist" in ds_name:
        arch = "preResNet18Norm01"
        ret = [
            f"./results/oos_repr/cwl2-128-{ds_name}-70-1.0-0.01-ce-vtor2-{arch}-0.0-2-adam-0-0.0.pkl",
            f"./results/oos_repr/cwl2-128-{ds_name}-70-0.5-0.01-advce-vtor2-{arch}-0.0-2-adam-0-0.0.pkl",
            f"./results/oos_repr/cwl2-128-{ds_name}-70-0.5-0.01-advce-vtor2-{arch}-0.0-2-adam-0-0.0.pkl",
            f"./results/oos_repr/cwl2-128-{ds_name}-70-0.5-0.01-trades6ce-vtor2-{arch}-0.0-2-adam-0-0.0.pkl",
            f"./results/oos_repr/cwl2-128-{ds_name}-70-0.5-0.01-trades6ce-vtor2-{arch}-0.0-2-adam-0-0.0.pkl",
            #f"./results/oos_repr/cwl2-128-{ds_name}-70-2.0-0.1-trades6ce-vtor2-ResNet18-0.0-2-adam-0-0.0.pkl",
        ]
    elif "dermamnist" in ds_name:
        arch = "preResNet18Norm01"
        ret = [
            f"./results/oos_repr/cwl2-128-{ds_name}-70-1.0-0.01-ce-vtor2-{arch}-0.0-2-adam-0-0.0.pkl",
            f"./results/oos_repr/cwl2-128-{ds_name}-70-1.0-0.01-advce-vtor2-{arch}-0.0-2-adam-0-0.0.pkl",
            f"./results/oos_repr/cwl2-128-{ds_name}-70-1.0-0.01-advce-vtor2-{arch}-0.0-2-adam-0-0.0.pkl",
            f"./results/oos_repr/cwl2-128-{ds_name}-70-2.0-0.01-trades6ce-vtor2-{arch}-0.0-2-adam-0-0.0.pkl",
            f"./results/oos_repr/cwl2-128-{ds_name}-70-2.0-0.01-trades6ce-vtor2-{arch}-0.0-2-adam-0-0.0.pkl",
            #f"./results/oos_repr/cwl2-128-{ds_name}-70-2.0-0.1-trades6ce-vtor2-ResNet18-0.0-2-adam-0-0.0.pkl",
        ]
    elif "mnist" in ds_name:
        ret = [
            f"./results/oos_repr/cwl2-128-{ds_name}-70-1.0-0.01-ce-vtor2-CNN002-0.9-2-sgd-0-0.0.pkl",
            #f"./results/oos_repr/cwl2-128-{ds_name}-70-1.0-0.01-mixupce-vtor2-CNN002-0.9-2-sgd-0-0.0.pkl",
            f"./results/oos_repr/cwl2-128-{ds_name}-70-2.0-0.01-advce-vtor2-CNN002-0.9-2-sgd-0-0.0.pkl",
            f"./results/oos_repr/cwl2-128-{ds_name}-70-2.0-0.01-trades6ce-vtor2-CNN002-0.9-2-sgd-0-0.0.pkl",
            f"./results/oos_repr/cwl2-128-{ds_name}-70-4.0-0.01-trades6ce-vtor2-CNN002-0.9-2-sgd-0-0.0.pkl",
            f"./results/oos_repr/cwl2-128-{ds_name}-70-8.0-0.01-trades6ce-vtor2-CNN002-0.9-2-sgd-0-0.0.pkl",
        ]
    elif "cifar10" in ds_name:
        ret = [
            f"./results/oos_repr/cwl2-64-{ds_name}-70-1.0-0.01-ce-vtor2-{arch_name}-0.0-2-adam-0-0.0.pkl",
            #f"./results/oos_repr/cwl2-64-{ds_name}-70-1.0-0.01-mixupce-vtor2-{arch_name}-0.0-2-adam-0-0.0.pkl",
            f"./results/oos_repr/cwl2-64-{ds_name}-70-2.0-0.01-advce-vtor2-{arch_name}-0.0-2-adam-0-0.0.pkl",
            f"./results/oos_repr/cwl2-64-{ds_name}-70-2.0-0.01-trades6ce-vtor2-{arch_name}-0.0-2-adam-0-0.0.pkl",
            f"./results/oos_repr/cwl2-64-{ds_name}-70-4.0-0.01-trades6ce-vtor2-{arch_name}-0.0-2-adam-0-0.0.pkl",
            f"./results/oos_repr/cwl2-64-{ds_name}-70-8.0-0.01-trades6ce-vtor2-{arch_name}-0.0-2-adam-0-0.0.pkl",
        ]
    elif "imgnet" in ds_name:
        ret = [
            f"./results/oos_repr/cwl2-128-{ds_name}-70-1.0-0.01-ce-vtor2-{arch_name}-0.0-2-adam-0-0.0.pkl",
            #f"./results/oos_repr/cwl2-128-{ds_name}-70-1.0-0.01-mixupce-vtor2-{arch_name}-0.0-2-adam-0-0.0.pkl",
            f"./results/oos_repr/cwl2-128-{ds_name}-70-2.0-0.01-advce-vtor2-{arch_name}-0.0-2-adam-0-0.0.pkl",
            f"./results/oos_repr/cwl2-128-{ds_name}-70-2.0-0.01-trades6ce-vtor2-{arch_name}-0.0-2-adam-0-0.0.pkl",
            f"./results/oos_repr/cwl2-128-{ds_name}-70-4.0-0.01-trades6ce-vtor2-{arch_name}-0.0-2-adam-0-0.0.pkl",
            f"./results/oos_repr/cwl2-128-{ds_name}-70-8.0-0.01-trades6ce-vtor2-{arch_name}-0.0-2-adam-0-0.0.pkl",
        ]
    return ret

In [12]:
datasets, ds_names, arch_names = [], [], []
accepted_models = ['natural', 'mixup', 'AT(2)', 'TRADES(2)', 'TRADES(4)', 'TRADES(8)']
accepted_models = ['natural', 'AT(2)', 'TRADES(2)', 'TRADES(4)', 'TRADES(8)']

#datasets += [f'pathmnistwo{i}' for i in [0, 1]]
#ds_names += [f"PathMNIST-wo{i}" for i in [0, 1]]
#arch_names += ["ResNet18"] * 2
#datasets += [f'dermamnistwo{i}' for i in [0,]]
#ds_names += [f"DermaMNIST-wo{i}" for i in [0,]]
#arch_names += ["preResNet50Norm01"] * 2

datasets += [f'mnistwo{i}' for i in range(10)]
ds_names += [f"MNIST-wo{i}" for i in range(10)]
arch_names += ["CNN002"] * 10

datasets += [f"cifar10wo{i}" for i in [0, 4, 9]]
ds_names += [f"CIFAR10-wo{i}" for i in [0, 4, 9]]
arch_names += ["WRN_40_10"] * 3

datasets += [f"cifar100coarsewo{i}" for i in [0, 4, 9]]
ds_names += [f"CIFAR100-wo{i}" for i in [0, 4, 9]]
arch_names += ["WRN_40_10"] * 3

datasets += [f"aug10-imgnet100wo{i}" for i in range(3)]
ds_names += [f"ImgNet100-wo{i}" for i in range(3)]
arch_names += ["ResNet50Norm01"] * 3

#accepted_models = ['natural', 'AT(2)', 'TRADES(2)']
#
#datasets += [f"cifar10wo{i}" for i in [0]]
#ds_names += [f"CIFAR10-wo{i}" for i in [0]]
#arch_names += ["DenseNet161"] * 1
#
#datasets += [f"cifar100coarsewo{i}" for i in [0]]
#ds_names += [f"CIFAR100-wo{i}" for i in [0]]
#arch_names += ["DenseNet161"] * 1

dfs = []
all_data = {}
for idx, ds_name in tqdm(enumerate(datasets), total=len(datasets)):
    paths = get_file_paths(ds_name, arch_names[idx])
    for i, path in enumerate(paths[:len(accepted_models)]):
        res2 = joblib.load(path.replace('./results/oos_repr', './results/get_preds'))
        tst_preds = res2['tst_pred']
        res = joblib.load(f"./out_of_sample/tst_k5/{ds_name}.pkl")
        tst_nn_preds = res['knn_y_2'][:, 0]
        indist_acc = (tst_preds == tst_nn_preds).mean()
        
        res = joblib.load(path)
        trn_acc = res['trn_acc']
        tst_acc = res['tst_acc']
        ood_preds = np.concatenate((res['oos_trn_pred'].reshape(-1), res['oos_tst_pred'].reshape(-1)))
        res = joblib.load(f"./out_of_sample/{ds_name}.pkl")
        nn_preds = np.concatenate((res['miss_trn_knn_y_2'].reshape(-1), res['miss_tst_knn_y_2'].reshape(-1)))
        
        ncg_acc = (ood_preds == nn_preds).mean()
        
        all_data[ds_names[idx], accepted_models[i]] = (trn_acc, tst_acc, ncg_acc, indist_acc)

  0%|          | 0/19 [00:00<?, ?it/s]

In [5]:

#accepted_models = ['natural', 'mixup', 'TRADES(2)', 'TRADES(4)', 'TRADES(8)', 'AT(2)']
accepted_models = ['natural', 'TRADES(2)', 'TRADES(4)', 'TRADES(8)', 'AT(2)']
for model in accepted_models:
    temp = []
    for ds_name in [f'MNIST-wo{i}' for i in range(10)]:
    #for ds_name in [f"CIFAR10-wo{i}" for i in [0, 4, 9]]:
    #for ds_name in [f"CIFAR100-wo{i}" for i in [0, 4, 9]]:
    #for ds_name in [f"ImgNet100-wo{i}" for i in range(3)]:
        #temp.append(all_data[(ds_name, model)][2])
        temp.append(all_data[(ds_name, model)][3])
    print(np.mean(temp), np.std(temp))

0.9720411841158546 0.0023383592799324096
0.9735498773516541 0.0023323235441087085
0.9580201170099913 0.00590466693901288
0.9729744993283349 0.0025041517542925777
0.9736533493524282 0.0023456230082861806


In [22]:
all_data.keys()

dict_keys([('MNIST-wo0', 'natural'), ('MNIST-wo0', 'mixup'), ('MNIST-wo0', 'AT(2)'), ('MNIST-wo0', 'TRADES(2)'), ('MNIST-wo0', 'TRADES(4)'), ('MNIST-wo1', 'natural'), ('MNIST-wo1', 'mixup'), ('MNIST-wo1', 'AT(2)'), ('MNIST-wo1', 'TRADES(2)'), ('MNIST-wo1', 'TRADES(4)'), ('MNIST-wo2', 'natural'), ('MNIST-wo2', 'mixup'), ('MNIST-wo2', 'AT(2)'), ('MNIST-wo2', 'TRADES(2)'), ('MNIST-wo2', 'TRADES(4)'), ('MNIST-wo3', 'natural'), ('MNIST-wo3', 'mixup'), ('MNIST-wo3', 'AT(2)'), ('MNIST-wo3', 'TRADES(2)'), ('MNIST-wo3', 'TRADES(4)'), ('MNIST-wo4', 'natural'), ('MNIST-wo4', 'mixup'), ('MNIST-wo4', 'AT(2)'), ('MNIST-wo4', 'TRADES(2)'), ('MNIST-wo4', 'TRADES(4)'), ('MNIST-wo5', 'natural'), ('MNIST-wo5', 'mixup'), ('MNIST-wo5', 'AT(2)'), ('MNIST-wo5', 'TRADES(2)'), ('MNIST-wo5', 'TRADES(4)'), ('MNIST-wo6', 'natural'), ('MNIST-wo6', 'mixup'), ('MNIST-wo6', 'AT(2)'), ('MNIST-wo6', 'TRADES(2)'), ('MNIST-wo6', 'TRADES(4)'), ('MNIST-wo7', 'natural'), ('MNIST-wo7', 'mixup'), ('MNIST-wo7', 'AT(2)'), ('MNI

In [25]:
ddff = pd.DataFrame.from_dict(all_data, orient="index", columns=['train acc.', 'test acc.', 'NCG score', 'in-dist NCG'])
ddff.index = pd.MultiIndex.from_tuples(ddff.index)
ddff = ddff.stack(0).unstack(1)[accepted_models]
text = ddff.to_latex(float_format="%.3f", multirow=True)
text = text.replace("DBS6(\$\\textbackslash lambda\$=1)", "ball")
text = text.replace("llrrrrr", "llccccc")
print(text)

\begin{tabular}{llccccc}
\toprule
          &             &  natural &  TRADES(2) &  TRADES(4) &  TRADES(8) &  AT(2) \\
\midrule
\multirow{4}{*}{CIFAR10-wo0} & train acc. &    1.000 &      0.992 &      0.870 &      0.878 &  0.999 \\
          & test acc. &    0.898 &      0.716 &      0.660 &      0.761 &  0.729 \\
          & NCG score &    0.355 &      0.492 &      0.520 &      0.483 &  0.494 \\
          & in-dist NCG &    0.357 &      0.410 &      0.418 &      0.381 &  0.403 \\
\cline{1-7}
\multirow{4}{*}{CIFAR10-wo4} & train acc. &    1.000 &      0.990 &      0.874 &      0.508 &  1.000 \\
          & test acc. &    0.886 &      0.742 &      0.700 &      0.485 &  0.754 \\
          & NCG score &    0.222 &      0.333 &      0.331 &      0.289 &  0.361 \\
          & in-dist NCG &    0.379 &      0.412 &      0.408 &      0.352 &  0.423 \\
\cline{1-7}
\multirow{4}{*}{CIFAR10-wo9} & train acc. &    1.000 &      0.992 &      0.948 &      0.778 &  1.000 \\
          & test acc. &    

## in- vs. out-of-distribution NCG score

In [15]:
ddff = pd.DataFrame.from_dict(all_data, orient="index", columns=['train acc.', 'test acc.', 'NCG score', 'in-dist NCG'])
ddff.index = pd.MultiIndex.from_tuples(ddff.index)
ddff = ddff.stack(0).unstack(1)[accepted_models]
text = ddff.to_latex(float_format="%.3f", multirow=True)
text = text.replace("DBS6(\$\\textbackslash lambda\$=1)", "ball")
text = text.replace("llrrrrr", "llccccc")
print(text)

\begin{tabular}{llccccc}
\toprule
          &             &  natural &  AT(2) &  TRADES(2) &  TRADES(4) &  TRADES(8) \\
\midrule
\multirow{4}{*}{CIFAR10-wo0} & train acc. &    1.000 &  0.999 &      0.992 &      0.870 &      0.878 \\
          & test acc. &    0.898 &  0.729 &      0.716 &      0.660 &      0.761 \\
          & NCG score &    0.355 &  0.494 &      0.492 &      0.520 &      0.483 \\
          & in-dist NCG &    0.357 &  0.403 &      0.410 &      0.418 &      0.381 \\
\cline{1-7}
\multirow{4}{*}{CIFAR10-wo4} & train acc. &    1.000 &  1.000 &      0.990 &      0.874 &      0.508 \\
          & test acc. &    0.886 &  0.754 &      0.742 &      0.700 &      0.485 \\
          & NCG score &    0.222 &  0.361 &      0.333 &      0.331 &      0.289 \\
          & in-dist NCG &    0.379 &  0.423 &      0.412 &      0.408 &      0.352 \\
\cline{1-7}
\multirow{4}{*}{CIFAR10-wo9} & train acc. &    1.000 &  1.000 &      0.992 &      0.948 &      0.778 \\
          & test acc. &    

In [19]:
results = {}
for model_name in accepted_models[1:]:
    tt = ddff[model_name] - ddff['natural']
    for ds_name in ds_names:
        print(ds_name)
        key = (ds_name.split('-')[0], model_name)
        print(tt.loc[(ds_name, 'NCG score')], tt.loc[(ds_name, 'in-dist NCG')])
        results[key] = results.setdefault(key, 0) + (
            tt.loc[(ds_name, 'NCG score')] < tt.loc[(ds_name, 'in-dist NCG')])
results

MNIST-wo0
0.09459655222367086 0.0012195121951219523
MNIST-wo1
0.28576869366510094 0.0019176536943034117
MNIST-wo2
0.14177396280400573 0.001784121320249743
MNIST-wo3
0.06483685758297153 0.0013348164627363657
MNIST-wo4
0.012162954279015259 0.002217786648924358
MNIST-wo5
0.08157769681609384 0.0009881422924901129
MNIST-wo6
0.03955788248981962 0.0018801150188011428
MNIST-wo7
0.13739201974496096 0.001003120820329939
MNIST-wo8
0.0712087912087912 0.001329492576999769
MNIST-wo9
0.13581488933601615 0.0024468913357802435
CIFAR10-wo0
0.13916666666666666 0.04533333333333334
CIFAR10-wo4
0.1391666666666667 0.04455555555555557
CIFAR10-wo9
0.06683333333333333 0.04655555555555557
CIFAR100-wo0
0.06533333333333333 0.03936842105263161
CIFAR100-wo4
0.05499999999999999 0.03452631578947368
CIFAR100-wo9
0.1313333333333333 0.03989473684210526
ImgNet100-wo0
0.011111111111111113 0.020606060606060607
ImgNet100-wo1
0.0022222222222222227 0.014141414141414142
ImgNet100-wo2
-0.0007407407407407397 0.00929292929292929
M

{('MNIST', 'AT(2)'): 0,
 ('CIFAR10', 'AT(2)'): 0,
 ('CIFAR100', 'AT(2)'): 0,
 ('ImgNet100', 'AT(2)'): 3,
 ('MNIST', 'TRADES(2)'): 0,
 ('CIFAR10', 'TRADES(2)'): 0,
 ('CIFAR100', 'TRADES(2)'): 0,
 ('ImgNet100', 'TRADES(2)'): 3,
 ('MNIST', 'TRADES(4)'): 2,
 ('CIFAR10', 'TRADES(4)'): 0,
 ('CIFAR100', 'TRADES(4)'): 0,
 ('ImgNet100', 'TRADES(4)'): 1,
 ('MNIST', 'TRADES(8)'): 2,
 ('CIFAR10', 'TRADES(8)'): 0,
 ('CIFAR100', 'TRADES(8)'): 0,
 ('ImgNet100', 'TRADES(8)'): 1}

In [17]:
tt

CIFAR10-wo0  train acc.     0.122489
             test acc.      0.137667
             NCG score     -0.128500
             in-dist NCG   -0.024111
CIFAR10-wo4  train acc.     0.491956
                              ...   
MNIST-wo8    in-dist NCG   -0.001219
MNIST-wo9    train acc.     0.002886
             test acc.      0.001891
             NCG score     -0.069848
             in-dist NCG   -0.001112
Length: 76, dtype: float64

In [ ]:
results = {}
for model_name in accepted_models[1:]:
    tt = ddff['natural'] - ddff[model_name]
    for ds_name in ds_names:
        key = (ds_name.split('-')[0], model_name)
        results[key] = results.setdefault(key, 0) + (tt.loc[(ds_name, 'in-dist NCG')] < 0)
results

### Chi-square

### t-test (rob vs. nat)

In [18]:
results = {}
for model_name in accepted_models[1:]:
    tt = ddff['natural'] - ddff[model_name]
    for ds_name in ds_names:
        key = (ds_name.split('-')[0], model_name)
        results[key] = results.setdefault(key, 0) + (tt.loc[(ds_name, 'NCG acc.')] < 0)
#pd.DataFrame.from_dict(results)
results

{('MNIST', 'mixup'): 5,
 ('CIFAR10', 'mixup'): 0,
 ('CIFAR100', 'mixup'): 2,
 ('ImgNet100', 'mixup'): 0,
 ('MNIST', 'TRADES(2)'): 10,
 ('CIFAR10', 'TRADES(2)'): 3,
 ('CIFAR100', 'TRADES(2)'): 3,
 ('ImgNet100', 'TRADES(2)'): 3,
 ('MNIST', 'TRADES(4)'): 7,
 ('CIFAR10', 'TRADES(4)'): 3,
 ('CIFAR100', 'TRADES(4)'): 3,
 ('ImgNet100', 'TRADES(4)'): 3,
 ('MNIST', 'TRADES(8)'): 8,
 ('CIFAR10', 'TRADES(8)'): 3,
 ('CIFAR100', 'TRADES(8)'): 3,
 ('ImgNet100', 'TRADES(8)'): 3,
 ('MNIST', 'AT(2)'): 10,
 ('CIFAR10', 'AT(2)'): 3,
 ('CIFAR100', 'AT(2)'): 3,
 ('ImgNet100', 'AT(2)'): 2}

### small NCG table

In [45]:
small_df = ddff.loc[[("MNIST-wo0", "NCG acc."), ("MNIST-wo9", "NCG acc."), ("CIFAR10-wo0", "NCG acc."), ("CIFAR100-wo0", "NCG acc."), ("ImgNet100-wo0", "NCG acc.")]]
small_df.index = small_df.index.droplevel(1)
text = small_df.transpose().to_latex(float_format="%.2f")
text = text.replace("MNIST-wo", "M-")
text = text.replace("CIFAR10-wo", "C10-")
text = text.replace("CIFAR100-wo", "C100-")
text = text.replace("ImgNet100-wo", "I-")
text = text.replace("0.", ".")
print(text)

\begin{tabular}{lrrrrr}
\toprule
{} &  M-0 &  M-9 &  C10-0 &  C100-0 &  I-0 \\
\midrule
natural   &       .39 &       .58 &         .35 &          .17 &           .03 \\
AT(2)     &       .46 &       .71 &         .49 &          .24 &           .04 \\
TRADES(2) &       .46 &       .69 &         .49 &          .25 &           .04 \\
TRADES(4) &       .48 &       .70 &         .52 &          .25 &           .05 \\
TRADES(8) &       .40 &       .66 &         .48 &          .21 &           .07 \\
\bottomrule
\end{tabular}



#### Merge feature space

In [27]:
feature_space_df_json = '{"natural":{"MNIST-wo0":0.2761118354,"MNIST-wo9":0.6632653061,"CIFAR10-wo0":0.7951666667,"CIFAR100-wo0":0.6326666667,"ImgNet100-wo0":0.0992592593},"AT(2)":{"MNIST-wo0":0.3221787629,"MNIST-wo9":0.7009198045,"CIFAR10-wo0":0.3866666667,"CIFAR100-wo0":0.0903333333,"ImgNet100-wo0":0.0155555556},"TRADES(2)":{"MNIST-wo0":0.3905548312,"MNIST-wo9":0.7053751078,"CIFAR10-wo0":0.8111666667,"CIFAR100-wo0":0.6923333333,"ImgNet100-wo0":0.122962963},"TRADES(4)":{"MNIST-wo0":0.4482109228,"MNIST-wo9":0.7298074159,"CIFAR10-wo0":0.8318333333,"CIFAR100-wo0":0.6786666667,"ImgNet100-wo0":0.1377777778},"TRADES(8)":{"MNIST-wo0":0.5769955092,"MNIST-wo9":0.7786720322,"CIFAR10-wo0":0.8258333333,"CIFAR100-wo0":0.6843333333,"ImgNet100-wo0":0.1214814815}}'

In [55]:
small_df2 = small_df.copy().transpose()
small_df2.columns = pd.MultiIndex.from_product([['pixel'], small_df2.columns.tolist()])
df2 = pd.DataFrame.from_dict(json.loads(feature_space_df_json)).transpose()
df2.columns = pd.MultiIndex.from_product([['feature'], df2.columns.tolist()])
merged_df = pd.concat((small_df2, df2), axis=1, )
text = merged_df.to_latex(float_format="%.2f", multicolumn=True, multicolumn_format="c")
text = text.replace("lrrrrrrrrrr", "lccccc|ccccc")
text = text.replace("MNIST-wo", "M-")
text = text.replace("CIFAR10-wo", "C10-")
text = text.replace("CIFAR100-wo", "C100-")
text = text.replace("ImgNet100-wo", "I-")
text = text.replace("0.", ".")
print(text)

\begin{tabular}{lcccccccccc}
\toprule
{} & \multicolumn{5}{c}{pixel} & \multicolumn{5}{c}{feature} \\
{} & M-0 & M-9 & C10-0 & C100-0 & I-0 & M-0 & M-9 & C10-0 & C100-0 & I-0 \\
\midrule
natural   &      .39 &      .58 &        .35 &         .17 &          .03 &      .28 &      .66 &        .80 &         .63 &          .10 \\
AT(2)     &      .46 &      .71 &        .49 &         .24 &          .04 &      .32 &      .70 &        .39 &         .09 &          .02 \\
TRADES(2) &      .46 &      .69 &        .49 &         .25 &          .04 &      .39 &      .71 &        .81 &         .69 &          .12 \\
TRADES(4) &      .48 &      .70 &        .52 &         .25 &          .05 &      .45 &      .73 &        .83 &         .68 &          .14 \\
TRADES(8) &      .40 &      .66 &        .48 &         .21 &          .07 &      .58 &      .78 &        .83 &         .68 &          .12 \\
\bottomrule
\end{tabular}

